In [126]:
import os
os.chdir('U:\\GitHubClones\\SALTSeniorDesign')
os.getcwd()
#import Excel
import xlwings as xw
import seaborn as sb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import plotly.plotly as py
import plotly
plotly.tools.set_credentials_file(username='amcdonne', api_key='TgHHDa2sn3SAj65ZH0dz')
#import py.plotly

In [1]:
import sys
!{sys.executable} -m pip install dash==0.21.0

  Using cached Flask-Compress-1.4.0.tar.gz
  Running setup.py bdist_wheel for dash: started
  Running setup.py bdist_wheel for dash: finished with status 'done'
  Stored in directory: C:\Users\amcdonne\AppData\Local\pip\Cache\wheels\5a\78\04\920dc004bf957455582d762c68e5611670e03182cb0831503f
  Running setup.py bdist_wheel for flask-compress: started
  Running setup.py bdist_wheel for flask-compress: finished with status 'done'
  Stored in directory: C:\Users\amcdonne\AppData\Local\pip\Cache\wheels\42\1e\83\8f3999abe10ae8f4f6463fd20e321b398d104c8c51b2401ca3
  Running setup.py bdist_wheel for plotly: started
  Running setup.py bdist_wheel for plotly: finished with status 'done'
  Stored in directory: C:\Users\amcdonne\AppData\Local\pip\Cache\wheels\35\e4\58\50b15ca81e61bd553172ca2dc2ddf4a53278da34f3a9d8e5e9
Successfully built dash flask-compress plotly


In [70]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [71]:
wb = xw.Book('Salt.xlsm')
setup = wb.sheets('Setup')
output = wb.sheets('Output')
strats = wb.sheets('Strat Tables')
performance = wb.sheets('Performance')
summ = wb.sheets('Summary')

# Path For the Current deal that is selected based on Setup
directory = setup['Setup.Directory'].value
shelf = setup['Setup.Shelf'].value
shelfList = setup['Shelf_List'].value
deal = setup['Setup.Deal'].value
report = setup['Setup.Report'].value
num_Deals = setup['Setup.Num_Deals'].value
max_Report = setup['Setup.Max_Report_Num'].value
#statsRange = summ['Summary.Statistics'].value

In [72]:
tapes = np.array(setup['Tapes'].value)
tapes = tapes[0:int(max_Report),0:int(num_Deals)]
dfTapes = pd.DataFrame(data=tapes[1:,1:], columns=tapes[0,1:])
dealList = list(dfTapes.columns)

In [136]:
cols = ['assetNumber', 'originationDate', 'reportingPeriodEndingDate','reportingPeriodBeginningDate',
        'originalLoanTerm','originalLoanAmount','vehicleValueAmount','originalInterestRatePercentage','subvented',
        'vehicleNewUsedCode', 'vehicleModelYear','vehicleTypeCode',
        'obligorCreditScore','obligorIncomeVerificationLevelCode', 'obligorEmploymentVerificationCode','obligorGeographicLocation',
        'paymentToIncomePercentage', 'reportingPeriodActualEndBalanceAmount',
        'reportingPeriodBeginningLoanBalanceAmount','totalActualAmountPaid',
        'remainingTermToMaturityNumber',
        'currentDelinquencyStatus','coObligorIndicator',
        'zeroBalanceCode']
stratDict = {
    'Fico Score':{
        'bin':[-1, 440, 500, 560, 620, 680, 740, 800, 1000],
        'label':['[-1, 440)', '[440, 500)', '[500, 560)', '[560, 620)', '[620, 680)', '[680, 740)', '[740, 800)', '[800, 1000)'],
        'index':['0 - 440', '441 - 500', '501 - 560', '561 - 620', '621 - 680', '681 - 740', '741 - 800', '801 +'],
        'histBin':100
    },
    'APR':{
        'bin':[0, .04, .08, .12, .16, .20, .24, .28, 1000],
        'label':['[0.0, 0.04)', '[0.04, 0.08)', '[0.08, 0.12)', '[0.12, 0.16)', '[0.16, 0.20)', '[0.20, 0.24)', '[0.24, 0.28)', '[0.28, 1000)'],
        'index': ['0.00% - 4.00%', '4.01% - 8.00%', '8.01% - 12.00%', '12.01% - 16.00%','16.01% - 20.00%', '20.01% - 24.00%', '24.01% - 28.00%', '28.00% +'],
        'histBin':75
    },
    'LTV':{
        'bin':[-1, .7, .8, .9, 1, 1.1, 1.2, 1.3, 1.4, 1000],
        'label':['[-1.0, 0.7)', '[0.7, 0.8)', '[0.8, 0.9)', '[0.9, 1.0)', '[1.0, 1.1)','[1.1, 1.2)', '[1.2, 1.3)', '[1.3, 1.4)', '[1.4, 1000)'],
        'index':['0.00% - 70.00%', '70.01% - 80.00%', '80.01% - 90.00%', '90.01% - 100.00%', '100.01% - 110.00%','110.01% - 120.00%', '120.01% - 130.00%', '130.01% - 140.00%', '140.01% +'],
        'histBin':100
        #['[-1.0, 0.7)', '[0.7, 0.8)', '[0.8, 0.9)', '[0.9, 1.0)', '[1.0, 1.1)','[1.1, 1.2)', '[1.2, 1.3)', '[1.3, 1.4)', '[1.4, 1000.0)']
    },
    'PTI':{
        'bin':[-1, .02, .04, .06, .08, .1, .12, .14, .16, 1000],
        'label':['[-1.0, 0.02)', '[0.02, 0.04)', '[0.04, 0.06)', '[0.06, 0.08)','[0.08, 0.1)', '[0.1, 0.12)', '[0.12, 0.14)', '[0.14, 0.16)', '[0.16, 1000)'],
        'index':['0.00% - 2.00%', '2.01% - 4.00%', '4.01% - 6.00%', '6.01% - 8.00%','8.01% - 10.00%', '10.01% - 12.00%', '12.01% - 14.00%','14.01% - 16.00%', '16.01% +'],
        'histBib':100
#['[-1.0, 0.02)', '[0.02, 0.04)', '[0.04, 0.06)', '[0.06, 0.08)','[0.08, 0.1)', '[0.1, 0.12)', '[0.12, 0.14)', '[0.14, 0.16)', '[0.16, 1000.0)']
    }
}

In [167]:
def stratTable(data):
    data  = data[data.zeroBalanceCode.isin(['-'])]
    WA = lambda x: np.average(x, axis=0, weights = data.loc[x.index, 'reportingPeriodActualEndBalanceAmount'])
    data['pctDeal'] = (data['reportingPeriodActualEndBalanceAmount']/(data['reportingPeriodActualEndBalanceAmount'].sum()))
    data['LTV'] = (data['originalLoanAmount']/data['vehicleValueAmount']) #Loan to value ratio
    
    a = OrderedDict([
    ('assetNumber',['count']),
    ('reportingPeriodActualEndBalanceAmount', ['sum','mean']),
    ('pctDeal', ['sum']),
    ('originalLoanTerm',[WA]),
    ('remainingTermToMaturityNumber',[WA]),
    ('paymentToIncomePercentage',[WA]),
    #('LTV',[WA]),
    ('originalInterestRatePercentage',[WA]),
    ('currentDelinquencyStatus',['mean'])
    ])
    
    keyList = ['Fico Score', 'APR', 'LTV', 'PTI']
    colsKeyList = list(a.keys())
    stratMetrics = ['obligorCreditScore', 'originalInterestRatePercentage', 'LTV', 'paymentToIncomePercentage']
    for i in range(0, len(stratMetrics)):
        data[stratMetrics[i]] = pd.to_numeric(data[stratMetrics[i]], errors='coerce')
        data[keyList[i]] = pd.cut(data[stratMetrics[i]], stratDict[keyList[i]]['bin'], right=True, labels=stratDict[keyList[i]]['label'])
        if stratMetrics[i] == 'obligorCreditScore':
            data[keyList[i]] = data[keyList[i]].cat.add_categories('No Score').fillna('No Score')
        
    return data
        #summ.range('j' + str(row)).options(pd.DataFrame, header=False).value = data[colsKeyList].groupby(by=[data[keyList[i]]]).agg(a)
        #stratTable = stratTable.reindex(keyList[i]['label'])

In [168]:
path = 'U:\\AutoLoanTapes\\Drive Auto Receivables Trust\\DRIVE 2017-1\\' + dfTapes[deal][0]
df = pd.read_csv(path, usecols=cols, memory_map=True, low_memory=False)
df = stratTable(df)
df.head(5)


,assetNumber,reportingPeriodBeginningDate,reportingPeriodEndingDate,originationDate,originalLoanAmount,originalLoanTerm,originalInterestRatePercentage,subvented,vehicleNewUsedCode,vehicleModelYear,vehicleTypeCode,vehicleValueAmount,obligorCreditScore,obligorIncomeVerificationLevelCode,obligorEmploymentVerificationCode,coObligorIndicator,paymentToIncomePercentage,obligorGeographicLocation,remainingTermToMaturityNumber,reportingPeriodBeginningLoanBalanceAmount,reportingPeriodActualEndBalanceAmount,totalActualAmountPaid,zeroBalanceCode,currentDelinquencyStatus,pctDeal,LTV,Fico Score,APR,PTI
0,"=""6345520""",05-01-2017,05-31-2017,10/2013,28203.21,72,0.1932,1,1,2013,3,23109.0,592.0,2,2,False,0.139531,MI,31,17115.35,16672.09,669.75,-,0,0.000012,"[1.2, 1.3)","[560, 620)","[0.16, 0.20)","[0.12, 0.14)"
1,"=""6345701""",05-01-2017,05-31-2017,11/2013,13786.00,72,0.2499,0,2,2009,1,11998.0,570.0,2,2,True,0.138530,SC,37,12184.17,12151.05,380.00,-,0,0.000009,"[1.1, 1.2)","[560, 620)","[0.24, 0.28)","[0.12, 0.14)"
2,"=""6345822""",05-01-2017,05-31-2017,11/2013,12515.00,60,0.2100,0,2,2003,3,11998.0,574.0,2,2,True,0.077623,CO,28,10288.10,10124.13,341.54,-,0,0.000007,"[1.0, 1.1)","[560, 620)","[0.20, 0.24)","[0.06, 0.08)"
3,"=""6346124""",05-01-2017,05-31-2017,10/2013,16699.00,60,0.2100,0,2,2007,2,15225.0,629.0,3,2,True,0.114215,OK,19,7828.79,7619.25,434.75,-,8,0.000006,"[1.0, 1.1)","[620, 680)","[0.20, 0.24)","[0.1, 0.12)"
4,"=""1006458""",05-01-2017,05-31-2017,05/2012,21404.16,72,0.2499,0,2,2006,3,17998.0,NaN,3,3,False,0.154751,NV,16,10385.66,10385.66,0.00,-,15,0.000008,"[1.1, 1.2)",No Score,"[0.24, 0.28)","[0.14, 0.16)"


In [172]:
#builds a dict that stores the dataframe for each month of the deal
deal = 'DRIVE 2017-1'
frame_storage = {}
strat_storage = {}
for i in range(5):
    path = 'U:\\AutoLoanTapes\\Drive Auto Receivables Trust\\DRIVE 2017-1\\' + dfTapes[deal][i]
    df = pd.read_csv(path, usecols=cols, memory_map=True, low_memory=False)
    df = stratTable(df)
    df.to_csv('C:\\Users\\amcdonne\\Downloads\\ABS\\' + 'Month_' + str(i+1) + '.csv', encoding='utf-8')
    #frame_storage['month_' + str(i+1)] = pd.read_csv(path,  memory_map=True, low_memory=False)
    #frame_storage['month_' + str(i+1)] = stratTable(frame_storage['month_' + str(i+1)], stratDict)
    

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

In [156]:
frame_storage['month_1']

,assetTypeNumber,assetNumber,reportingPeriodBeginningDate,reportingPeriodEndingDate,originatorName,originationDate,originalLoanAmount,originalLoanTerm,loanMaturityDate,originalInterestRatePercentage,interestCalculationTypeCode,originalInterestRateTypeCode,originalInterestOnlyTermNumber,originalFirstPaymentDate,underwritingIndicator,gracePeriodNumber,paymentTypeCode,subvented,vehicleManufacturerName,vehicleModelName,vehicleNewUsedCode,vehicleModelYear,vehicleTypeCode,vehicleValueAmount,vehicleValueSourceCode,obligorCreditScoreType,obligorCreditScore,obligorIncomeVerificationLevelCode,obligorEmploymentVerificationCode,coObligorIndicator,paymentToIncomePercentage,obligorGeographicLocation,assetAddedIndicator,remainingTermToMaturityNumber,reportingPeriodModificationIndicator,servicingAdvanceMethodCode,reportingPeriodBeginningLoanBalanceAmount,nextReportingPeriodPaymentAmountDue,reportingPeriodInterestRatePercentage,nextInterestRatePercentage,servicingFeePercentage,servicingFlatFeeAmount,otherServicerFeeRetainedByServicer,otherAssessedUncollectedServicerFeeAmount,scheduledInterestAmount,scheduledPrincipalAmount,otherPrincipalAdjustmentAmount,reportingPeriodActualEndBalanceAmount,reportingPeriodScheduledPaymentAmount,totalActualAmountPaid,actualInterestCollectedAmount,actualPrincipalCollectedAmount,actualOtherCollectedAmount,servicerAdvancedAmount,interestPaidThroughDate,zeroBalanceEffectiveDate,zeroBalanceCode,currentDelinquencyStatus,primaryLoanServicerName,mostRecentServicingTransferReceivedDate,assetSubjectDemandIndicator,assetSubjectDemandStatusCode,repurchaseAmount,DemandResolutionDate,repurchaserName,repurchaseReplacementReasonCode,chargedoffPrincipalAmount,recoveredAmount,modificationTypeCode,paymentExtendedNumber,repossessedIndicator,repossessedProceedsAmount
0,SC,"=""6345520""",05-01-2017,05-31-2017,SC,10/2013,28203.21,72,11/2019,0.1932,1,1,-,12/2013,True,2,2,1,DODGE TRUCK,Journey,1,2013,3,23109.00,98,Bureau,592,2,2,False,0.139531,MI,-,31,False,1,17115.35,1586.34,0.1932,0.1932,0.0,-,-,0.00,256.2709,408.2091,0.0,16672.09,664.48,669.75,226.49,443.26,0.00,-,05-04-2017,-,-,0,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,False,-
1,SC,"=""6345701""",05-01-2017,05-31-2017,SC,11/2013,13786.00,72,05/2020,0.2499,1,1,-,12/2013,True,1,2,0,FORD,Focus,2,2009,1,11998.00,98,Bureau,570,2,2,True,0.138530,SC,-,37,False,1,12184.17,499.93,0.2499,0.2499,0.0,-,-,365.42,173.2527,198.0173,0.0,12151.05,371.27,380.00,342.02,33.12,4.86,-,05-17-2017,-,-,0,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,False,-
2,SC,"=""6345822""",05-01-2017,05-31-2017,SC,11/2013,12515.00,60,08/2019,0.2100,1,1,-,12/2013,True,1,2,0,GMC LIGHT DUTY,Yukon,2,2003,3,11998.00,98,Bureau,574,2,2,True,0.077623,CO,-,28,False,1,10288.10,583.43,0.2100,0.2100,0.0,-,-,105.00,92.9365,245.6335,0.0,10124.13,338.57,341.54,177.57,163.97,0.00,-,05-15-2017,-,-,0,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,False,-
3,SC,"=""6346124""",05-01-2017,05-31-2017,SC,10/2013,16699.00,60,11/2018,0.2100,1,1,-,12/2013,True,2,2,0,TOYOTA,Tundra,2,2007,2,15225.00,98,Bureau,629,3,2,True,0.114215,OK,-,19,False,1,7828.79,990.21,0.2100,0.2100,0.0,-,-,22.50,124.0593,327.7007,0.0,7619.25,451.76,434.75,225.21,209.54,0.00,-,05-14-2017,-,-,8,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,False,-
4,SC,"=""1006458""",05-01-2017,05-31-2017,SC,05/2012,21404.16,72,08/2018,0.2499,1,1,-,06/2012,True,1,2,0,JEEP,Grand Cherokee,2,2006,3,17998.00,98,Bureau,-,3,3,False,0.154751,NV,-,16,False,1,10385.66,1277.14,0.2499,0.2499,0.0,-,-,429.72,130.9025,445.5375,0.0,10385.66,576.44,0.00,0.00,0.00,0.00,-,04-30-2017,-,-,15,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,True,0.00
5,SC,"=""1006584""",05-01-2017,05-31-2017,SC,05/2012,18099.08,72,09/2018,0.2499,1,1,-,06/2012,True,1,2,0,FORD TRUCK,Escape,2,2008,3,16849.00,98,Bureau,549,3,3,True,0.147130,MS,-,17,False,1,9504.84,1037.74,0.2499,0.2499,0.0,-,-,105.00,110.6920,376.7380,0.0,9128.09,487.43,585.00,208.25,376.75,0.00,-,05-23-2017,-,-,0,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,False,-
6,SC,"=""1007110""",05-01-2017,05-31-2017,SC,05/2012,15276.82,72,08/2018

In [128]:
frame_storage['month_1'] = frame_storage['month_1'][~frame_storage['month_1'].obligorCreditScore.isin(['-'])]
frame_storage['month_1'].head()


,assetTypeNumber,assetNumber,reportingPeriodBeginningDate,reportingPeriodEndingDate,originatorName,originationDate,originalLoanAmount,originalLoanTerm,loanMaturityDate,originalInterestRatePercentage,interestCalculationTypeCode,originalInterestRateTypeCode,originalInterestOnlyTermNumber,originalFirstPaymentDate,underwritingIndicator,gracePeriodNumber,paymentTypeCode,subvented,vehicleManufacturerName,vehicleModelName,vehicleNewUsedCode,vehicleModelYear,vehicleTypeCode,vehicleValueAmount,vehicleValueSourceCode,obligorCreditScoreType,obligorCreditScore,obligorIncomeVerificationLevelCode,obligorEmploymentVerificationCode,coObligorIndicator,paymentToIncomePercentage,obligorGeographicLocation,assetAddedIndicator,remainingTermToMaturityNumber,reportingPeriodModificationIndicator,servicingAdvanceMethodCode,reportingPeriodBeginningLoanBalanceAmount,nextReportingPeriodPaymentAmountDue,reportingPeriodInterestRatePercentage,nextInterestRatePercentage,servicingFeePercentage,servicingFlatFeeAmount,otherServicerFeeRetainedByServicer,otherAssessedUncollectedServicerFeeAmount,scheduledInterestAmount,scheduledPrincipalAmount,otherPrincipalAdjustmentAmount,reportingPeriodActualEndBalanceAmount,reportingPeriodScheduledPaymentAmount,totalActualAmountPaid,actualInterestCollectedAmount,actualPrincipalCollectedAmount,actualOtherCollectedAmount,servicerAdvancedAmount,interestPaidThroughDate,zeroBalanceEffectiveDate,zeroBalanceCode,currentDelinquencyStatus,primaryLoanServicerName,mostRecentServicingTransferReceivedDate,assetSubjectDemandIndicator,assetSubjectDemandStatusCode,repurchaseAmount,DemandResolutionDate,repurchaserName,repurchaseReplacementReasonCode,chargedoffPrincipalAmount,recoveredAmount,modificationTypeCode,paymentExtendedNumber,repossessedIndicator,repossessedProceedsAmount
0,SC,"=""6345520""",05-01-2017,05-31-2017,SC,10/2013,28203.21,72,11/2019,0.1932,1,1,-,12/2013,True,2,2,1,DODGE TRUCK,Journey,1,2013,3,23109.0,98,Bureau,592,2,2,False,0.139531,MI,-,31,False,1,17115.35,1586.34,0.1932,0.1932,0.0,-,-,0.00,256.2709,408.2091,0.0,16672.09,664.48,669.75,226.49,443.26,0.00,-,05-04-2017,-,-,0,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,False,-
1,SC,"=""6345701""",05-01-2017,05-31-2017,SC,11/2013,13786.00,72,05/2020,0.2499,1,1,-,12/2013,True,1,2,0,FORD,Focus,2,2009,1,11998.0,98,Bureau,570,2,2,True,0.138530,SC,-,37,False,1,12184.17,499.93,0.2499,0.2499,0.0,-,-,365.42,173.2527,198.0173,0.0,12151.05,371.27,380.00,342.02,33.12,4.86,-,05-17-2017,-,-,0,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,False,-
2,SC,"=""6345822""",05-01-2017,05-31-2017,SC,11/2013,12515.00,60,08/2019,0.2100,1,1,-,12/2013,True,1,2,0,GMC LIGHT DUTY,Yukon,2,2003,3,11998.0,98,Bureau,574,2,2,True,0.077623,CO,-,28,False,1,10288.10,583.43,0.2100,0.2100,0.0,-,-,105.00,92.9365,245.6335,0.0,10124.13,338.57,341.54,177.57,163.97,0.00,-,05-15-2017,-,-,0,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,False,-
3,SC,"=""6346124""",05-01-2017,05-31-2017,SC,10/2013,16699.00,60,11/2018,0.2100,1,1,-,12/2013,True,2,2,0,TOYOTA,Tundra,2,2007,2,15225.0,98,Bureau,629,3,2,True,0.114215,OK,-,19,False,1,7828.79,990.21,0.2100,0.2100,0.0,-,-,22.50,124.0593,327.7007,0.0,7619.25,451.76,434.75,225.21,209.54,0.00,-,05-14-2017,-,-,8,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,False,-
5,SC,"=""1006584""",05-01-2017,05-31-2017,SC,05/2012,18099.08,72,09/2018,0.2499,1,1,-,06/2012,True,1,2,0,FORD TRUCK,Escape,2,2008,3,16849.0,98,Bureau,549,3,3,True,0.147130,MS,-,17,False,1,9504.84,1037.74,0.2499,0.2499,0.0,-,-,105.00,110.6920,376.7380,0.0,9128.09,487.43,585.00,208.25,376.75,0.00,-,05-23-2017,-,-,0,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,False,-


In [129]:
pv1 = frame_storage['month_1'][['originationDate', 'originalLoanAmount']]
pv1.head()

,originationDate,originalLoanAmount
0,10/2013,28203.21
1,11/2013,13786.00
2,11/2013,12515.00
3,10/2013,16699.00
5,05/2012,18099.08


In [133]:
pv2 = pd.pivot_table(frame_storage['month_1'], values='originalLoanAmount', index = 'originationDate', aggfunc = np.sum)
pv2.head()

,originalLoanAmount
originationDate,
01/2011,111514.22
01/2012,120306.94
01/2013,22154.25
01/2014,1298179.99
01/2015,102080.66


In [135]:
trace1 = go.Scatter(x=pv2.index, y=pv2.originalLoanAmount, name='Original Balance', mode = 'markers')
#trace2 =  go.Bar(x=pv2.index, y=pv2['reportingPeriodActualEndBalanceAmount'], name='Current Balance')
data=go.Data([trace1])
layout=go.Layout(title="First Plot", xaxis={'title':'x1'}, yaxis={'title':'x2'})
figure=go.Figure(data=data,layout=layout)
py.iplot(figure, filename='pyguide_1')
trace1

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~amcdonne/0 or inside your plot.ly account where it is named 'pyguide_1'


{'mode': 'markers',
 'name': 'Original Balance',
 'type': 'scatter',
 'x': Index([u'01/2011', u'01/2012', u'01/2013', u'01/2014', u'01/2015', u'01/2016', u'01/2017', u'02/2011', u'02/2012', u'02/2013', u'02/2014', u'02/2015', u'02/2016', u'02/2017', u'03/2011', u'03/2012', u'03/2013', u'03/2014', u'03/2015', u'03/2016', u'03/2017', u'04/2011', u'04/2012', u'04/2013', u'04/2014', u'04/2015', u'04/2016', u'04/2017', u'05/2011', u'05/2012', u'05/2013', u'05/2014', u'05/2015', u'05/2016', u'05/2017', u'06/2011', u'06/2012', u'06/2013', u'06/2014', u'06/2015', u'06/2016', u'07/2011', u'07/2012', u'07/2013', u'07/2014', u'07/2015', u'07/2016', u'08/2011', u'08/2012', u'08/2013', u'08/2014', u'08/2015', u'08/2016', u'09/2011', u'09/2012', u'09/2013', u'09/2014', u'09/2015', u'09/2016', u'10/2010', u'10/2011', u'10/2013', u'10/2014', u'10/2015', u'10/2016', u'11/2010', u'11/2011', u'11/2012', u'11/2013', u'11/2015', u'11/2016', u'12/2010', u'12/2011', u'12/2013', u'12/2014', u'12/2015', u'12/2

In [120]:
app = dash.Dash()

app.layout = html.Div(children=[
    html.H1(children='ABS Credit Metrics'),
    html.Div(children='''SALT'''),
    dcc.Graph(
        id='example-graph',
        figure={
            'data': trace1,
            'layout':
            go.Layout()
        })
])


In [121]:
if __name__ == '__main__':
    app.run_server(debug=True)

 * Restarting with stat


SystemExit: 1

In [66]:
app = dash.Dash()

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(
        children='Hello Dash',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),

    html.Div(children='Dash: A web application framework for Python.', style={
        'textAlign': 'center',
        'color': colors['text']
    }),

    dcc.Graph(
        id='example-graph-2',
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
            ],
            'layout': {
                'plot_bgcolor': colors['background'],
                'paper_bgcolor': colors['background'],
                'font': {
                    'color': colors['text']
                }
            }
        }
    )
])


In [ ]:
if __name__ == '__main__':
    app.run_server()